In [ ]:
import warnings
warnings.filterwarnings('ignore',category=RuntimeWarning)
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import glob,os,sys
from tqdm import tqdm
import datetime
import proplot as plot
from netCDF4 import Dataset
from wrf import getvar
import celer
%matplotlib inline

sys.path.insert(1, '/work/08350/tg876493/stampede2/python_codes/2020_TC_CRF/dev/freddy0218/tools/')
import derive_var
import read_and_proc

plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})

In [2]:
############################################################################################################################################
# PCA_dict: Processed EOF objects; A_dict: Variable anomaly matrices; Af_dict: Flattened variable anomaly matrices
# *note that following EOF conventions, we derived EOFs with anomalies.
############################################################################################################################################
import pickle
path = './output/1010_ns/'
filePCA = [path+'dtheta_pca_ns',path+'urad_pca_ns',path+'vtan_pca_ns',path+'w_pca_ns',path+'uradR_pca_ns',path+'vtanR_pca_ns',path+'qv_pca_ns']
fileA = [path+'normalT_ctrl_ns',path+'normalUrad_ctrl_ns',path+'normalVtan_ctrl_ns',path+'normalw_ctrl_ns',path+'normalUradR_ctrl_ns',path+'normalVtanR_ctrl_ns',path+'normalQV_ctrl_ns']
fileAf = [path+'normalT_ctrlf_ns',path+'normalUrad_ctrlf_ns',path+'normalVtan_ctrlf_ns',path+'normalw_ctrlf_ns',path+'normalUradR_ctrlf_ns',path+'normalVtanR_ctrlf_ns',path+'normalQV_ctrlf_ns']

outputPCA,outputA,outputAf = [],[],[]
for i in range(len(filePCA)):
    with open(filePCA[i],'rb') as f:
        outputPCA.append(pickle.load(f))
    with open(fileA[i],'rb') as f:
        outputA.append(pickle.load(f))
    with open(fileAf[i],'rb') as f:
        outputAf.append(pickle.load(f))

PCA_dict = {'dtheta':outputPCA[0],'u':outputPCA[1],'v':outputPCA[2],'w':outputPCA[3],'ur':outputPCA[4],'vr':outputPCA[5],'qv':outputPCA[6]}
#A_dict = {'dtheta':outputA[0],'u':outputA[1],'v':outputA[2],'w':outputA[3],'ur':outputA[4],'vr':outputA[5],'qv':outputA[6]}
Af_dict = {'dtheta':outputAf[0],'u':outputAf[1],'v':outputAf[2],'w':outputAf[3],'ur':outputAf[4],'vr':outputAf[5],'qv':outputAf[6]}

/work/08350/tg876493/stampede2/gpu_env/fred_workenv/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator PCA from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### WRF

In [3]:
#Function####################################################
import json,pickle
def nearest_index(array, value):
    idx = (np.abs(array-value)).argmin()
    return idx.values
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

def do_PCA(var=None,timezoom=None,smooth='Yes',gaussian=0.9,fromcenter='Yes',inradius=None,outradius=None,donormal='Yes',doPCA='Yes'):
    """
    Var: Input variable (must have 4 dimensions! Time-pres-theta-radius)
    """
    if smooth=='Yes':
        from scipy.ndimage import gaussian_filter
        normal_var = []
        if donormal=='Yes':
            for presindex in range(len(var[0,:,0,0])):
                normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'Yes'),sigma=gaussian))
        elif donormal=='No':
            for presindex in range(len(var[0,:,0,0])):
                normal_var.append(gaussian_filter(normalize_inner(var[:,presindex,:,:],outradius,'No'),sigma=gaussian))            
        normal_var = np.swapaxes(np.asarray(normal_var),0,1)
    else:
        normal_var = []
        for presindex in range(len(var[0,:,0,0])):
            normal_var.append(normalize_inner(var[:,presindex,:,:],outradius))
        normal_var = np.swapaxes(np.asarray(normal_var),0,1)
    if fromcenter=='Yes':
        normal_varf = np.asarray([normal_var[i,:,:,:outradius].flatten() \
                                  for i in range(len(normal_var[timezoom[0]:timezoom[1],0,0,0]))])
    elif fromcenter=='No':
        normal_varf = np.asarray([normal_var[i,:,:,inradius:outradius].flatten() \
                                  for i in range(len(normal_var[timezoom[0]:timezoom[1],0,0,0]))])        
    print("--Finish preprocesing--")
    if doPCA=='Yes':
        from sklearn.decomposition import PCA
        import time
        start_time = time.time()
        skpcaVAR = PCA()
        skpcaVAR.fit(normal_varf.copy())
        print("--- %s seconds ---" % (time.time() - start_time))
        return skpcaVAR,normal_var,normal_varf
    else:
        return normal_var,normal_varf

def forward_diff(arrayin=None,delta=None,axis=None,LT=1):
    result = []
    if axis==0:
        if LT==1:
            for i in range(0,arrayin.shape[axis]-1):
                temp = (arrayin[i+1,:]-arrayin[i,:])/(LT*delta)
                result.append(temp)
            return np.asarray(result)
        elif LT==2:
            for i in range(0,arrayin.shape[axis]-2):
                temp = (arrayin[i+2,:]-arrayin[i,:])/(LT*delta)
                result.append(temp)
            return np.asarray(result)
        elif LT==3:
            for i in range(0,arrayin.shape[axis]-3):
                temp = (arrayin[i+3,:]-arrayin[i,:])/(LT*delta)
                result.append(temp)
            return np.asarray(result)
        
def normalize_inner(var=None,outerradius=None,standard='Yes'):
    PWper_ctrl = []
    for indx in range(len(var[:,0,0])):
        temp = var[indx,:,:outerradius]-np.nanmean(var[indx,:,:outerradius],axis=(0,1))
        if standard=='Yes':
            PWper_ctrl.append((temp-np.nanmean(temp))/np.nanstd(temp))
        elif standard=='No':
            PWper_ctrl.append((temp))
    del temp
    return np.asarray(PWper_ctrl)

In [4]:
origin_path = '/scratch/06040/tg853394/tc/output/redux/maria/'

#############################################################
ctrlvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ctl/azim_W_0-144hr.nc',origin_path+'ctl/azim_QVAPOR_0-144hr.nc',
                                                              origin_path+'ctl/azim_RTHRATLW_0-144hr.nc',origin_path+'ctl/azim_RTHRATSW_0-144hr.nc'],fieldname=['W','QVAPOR','RTHRATLW','RTHRATSW'])
ncrf60var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ncrf_60h/azim_W_60-144hr.nc',origin_path+'ncrf_60h/azim_QVAPOR_60-144hr.nc',
                                                                origin_path+'ncrf_60h/azim_RTHRATLW_60-144hr.nc',origin_path+'ncrf_60h/azim_RTHRATSW_60-144hr.nc'],fieldname=['W','QVAPOR','RTHRATLW','RTHRATSW'])
ncrf36var_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'ncrf_36h/azim_W_36-144hr.nc',origin_path+'ncrf_36h/azim_QVAPOR_36-144hr.nc',
                                                                origin_path+'ncrf_36h/azim_RTHRATLW_36-144hr.nc',origin_path+'ncrf_36h/azim_RTHRATSW_36-144hr.nc'],fieldname=['W','QVAPOR','RTHRATLW','RTHRATSW'])
lwcrfvar_dict = read_and_proc.read_some_azimuth_fields(fileloc=[origin_path+'lwcrf/azim_W_36-144hr.nc',origin_path+'lwcrf/azim_QVAPOR_36-144hr.nc',
                                                               origin_path+'lwcrf/azim_RTHRATLW_36-144hr.nc',origin_path+'lwcrf/azim_RTHRATSW_36-144hr.nc'],fieldname=['W','QVAPOR','RTHRATLW','RTHRATSW'])

path = '/work2/08350/tg876493/stampede2/python_codes/tcg_radiation/output/'
file = [path+'ctrl_thetaall',path+'ncrf36_thetaall',path+'ncrf60_thetaall',path+'lwcrf_thetaall']
output = []
for i in range(len(file)):
    with open(file[i],'rb') as f:
        output.append(pickle.load(f))
theta_dict = {'CTRL_theta':output[0],'NCRF36_theta':output[1],'NCRF60_theta':output[2],'LWCRF_theta':output[3]}
# To be consistent with azimuthal files
########################################################################################################
ctrl_thetaA,ncrf36_thetaA,ncrf60_thetaA,lwcrf_thetaA = np.swapaxes(np.asarray(theta_dict['CTRL_theta']),0,1),\
np.swapaxes(np.asarray(theta_dict['NCRF36_theta']),0,1),np.swapaxes(np.asarray(theta_dict['NCRF60_theta']),0,1),np.swapaxes(np.asarray(theta_dict['LWCRF_theta']),0,1)
del output,theta_dict,outputPCA,outputAf

In [5]:
from scipy.ndimage import gaussian_filter
###################################################################
# Cut off time
###################################################################
ctrl_t2,ctrl_t4 = nearest_index(ctrlvar_dict['W']['W'].time/24,1)-1,nearest_index(ctrlvar_dict['W']['W'].time/24,5)-1
###################################################################
# Heating Rate
###################################################################
ncrf36_thetaA_n,ncrf60_thetaA_n,lwcrf_thetaA_n = read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,ncrf36_thetaA,'NCRF36','Yes'),\
read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,ncrf60_thetaA,'NCRF60','Yes'),read_and_proc.add_ctrl_before_senstart(ctrl_thetaA,lwcrf_thetaA,'NCRF36','Yes')

ctrl_thetagrad_b,ncrf36_thetagrad_b,ncrf60_thetagrad_b,lwcrf_thetagrad_b = forward_diff(gaussian_filter(ctrl_thetaA,sigma=[3,0,0,0]),60*60,0),forward_diff(gaussian_filter(ncrf36_thetaA_n,sigma=[3,0,0,0]),60*60,0),\
forward_diff(gaussian_filter(ncrf60_thetaA_n,sigma=[3,0,0,0]),60*60,0),forward_diff(gaussian_filter(lwcrf_thetaA_n,sigma=[3,0,0,0]),60*60,0)

def proc_invar_forPCA(var=None,timezoom=[ctrl_t2,ctrl_t4],smooth='Yes',gaussian=[0,0,0],fromcenter='Yes',inradius=None,outradius=None,donormal='No',doPCA='No'):
    return do_PCA(var=var,timezoom=[ctrl_t2,ctrl_t4],smooth='Yes',gaussian=[0,0,0],fromcenter='Yes',inradius=None,outradius=None,donormal='No',doPCA='No')

### Radiative Heating + Residual Heating

In [6]:
del ctrl_thetaA,ncrf36_thetaA,ncrf60_thetaA
rad_ctrl = gaussian_filter(ctrlvar_dict['RTHRATLW']['RTHRATLW']+ctrlvar_dict['RTHRATSW']['RTHRATSW'],[3,0,0,0])
rad_ncrf36 = read_and_proc.add_ctrl_before_senstart(rad_ctrl,gaussian_filter(ncrf36var_dict['RTHRATLW']['RTHRATLW']+ncrf36var_dict['RTHRATSW']['RTHRATSW'],[3,0,0,0]),'NCRF36','Yes')
rad_ncrf60 = read_and_proc.add_ctrl_before_senstart(rad_ctrl,gaussian_filter(ncrf60var_dict['RTHRATLW']['RTHRATLW']+ncrf60var_dict['RTHRATSW']['RTHRATSW'],[3,0,0,0]),'NCRF60','Yes')
rad_lwcrf = read_and_proc.add_ctrl_before_senstart(rad_ctrl,gaussian_filter(lwcrfvar_dict['RTHRATLW']['RTHRATLW']+lwcrfvar_dict['RTHRATSW']['RTHRATSW'],[3,0,0,0]),'NCRF36','Yes')

resid_ctrl = ctrl_thetagrad_b-rad_ctrl[1:]
resid_ncrf36 = ncrf36_thetagrad_b-rad_ncrf36[1:]
resid_ncrf60 = ncrf60_thetagrad_b-rad_ncrf60[1:]
resid_lwcrf = lwcrf_thetagrad_b-rad_lwcrf[1:]
del ncrf36var_dict,ncrf60var_dict,lwcrfvar_dict

In [7]:
# Heating Rate
#########################################################################################################
r0,r200=(nearest_index(ctrlvar_dict['W']['W'].radius,0),nearest_index(ctrlvar_dict['W']['W'].radius,200))
r300,r500=(nearest_index(ctrlvar_dict['W']['W'].radius,300),nearest_index(ctrlvar_dict['W']['W'].radius,500))
########################################################################################################################
# Patch data
########################################################################################################################
#_,normalWncrf36f = proc_invar_forPCA(read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['W']['W'],ncrf36var_dict['W']['W'],'NCRF36','Yes'),[ctrl_t2,ctrl_t4],'Yes',[0,0,0],'Yes',None,r500,'No','No')
#_,normalWncrf60f = proc_invar_forPCA(read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['W']['W'],ncrf60var_dict['W']['W'],'NCRF60','Yes'),[ctrl_t2,ctrl_t4],'Yes',[0,0,0],'Yes',None,r500,'No','No')
#_,normalWlwcrff = proc_invar_forPCA(read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['W']['W'],lwcrfvar_dict['W']['W'],'NCRF36','Yes'),[ctrl_t2,ctrl_t4],'Yes',[0,0,0],'Yes',None,r500,'No','No')

_,normalRADctrlf = proc_invar_forPCA(rad_ctrl,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')
_,normalRADncrf36f = proc_invar_forPCA(rad_ncrf36,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')
_,normalRADncrf60f = proc_invar_forPCA(rad_ncrf60,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')
_,normalRADlwcrff= proc_invar_forPCA(rad_lwcrf,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')

_,normalRESIctrlf = proc_invar_forPCA(resid_ctrl,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')
_,normalRESIncrf36f = proc_invar_forPCA(resid_ncrf36,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')
_,normalRESIncrf60f = proc_invar_forPCA(resid_ncrf60,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')
_,normalRESIlwcrff = proc_invar_forPCA(resid_lwcrf,[ctrl_t2,ctrl_t4],'Yes',[3,0,0],'Yes',None,r500,'No','No')

#_,normalQVncrf36f = proc_invar_forPCA(read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['QVAPOR']['QVAPOR'],ncrf36var_dict['QVAPOR']['QVAPOR'],'NCRF36','Yes'),[ctrl_t2,ctrl_t4],'Yes',[0,0,0],'Yes',None,r500,'No','No')
#_,normalQVncrf60f = proc_invar_forPCA(read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['QVAPOR']['QVAPOR'],ncrf60var_dict['QVAPOR']['QVAPOR'],'NCRF60','Yes'),[ctrl_t2,ctrl_t4],'Yes',[0,0,0],'Yes',None,r500,'No','No')
#_,normalQVlwcrff = proc_invar_forPCA(read_and_proc.add_ctrl_before_senstart(ctrlvar_dict['QVAPOR']['QVAPOR'],lwcrfvar_dict['QVAPOR']['QVAPOR'],'NCRF36','Yes'),[ctrl_t2,ctrl_t4],'Yes',[0,0,0],'Yes',None,r500,'No','No')

del rad_ctrl,rad_ncrf36,rad_ncrf60,rad_lwcrf,resid_ctrl,resid_ncrf36,resid_ncrf60,resid_lwcrf

MemoryError: Unable to allocate 2.17 GiB for an array with shape (96, 6079320) and data type float32

#### Save to pickle

In [8]:
def save_to_pickle(loc=None,var=None):
    with open(loc,"wb") as f:
        pickle.dump(var,f)
    return None

save_to_pickle('./output/1010_ns/sensitivity/RAD_ctrlf_ns',rad_ctrl)
save_to_pickle('./output/1010_ns/sensitivity/RAD_ncrf36f_ns',rad_ncrf36)
save_to_pickle('./output/1010_ns/sensitivity/RAD_ncrf60f_ns',rad_ncrf60)
save_to_pickle('./output/1010_ns/sensitivity/RAD_lwcrff_ns',rad_lwcrf)

save_to_pickle('./output/1010_ns/sensitivity/RESI_ctrlf_ns',resid_ctrl)
save_to_pickle('./output/1010_ns/sensitivity/RESI_ncrf36f_ns',resid_ncrf36)
save_to_pickle('./output/1010_ns/sensitivity/RESI_ncrf60f_ns',resid_ncrf60)
save_to_pickle('./output/1010_ns/sensitivity/RESI_lwcrff_ns',resid_lwcrf)